In [6]:
import os

from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import multiprocessing as mp
import shutil

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras import regularizers

import pandas as pd
import numpy as np
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

In [7]:
img_size = 96
epochs = 75
batch_size = 48
testsplit = .2
targetx = 96
targety = 96
learning_rate = 0.0001
classes = 7
seed = 23
print(seed)

23


In [8]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
    validation_split=testsplit,
        preprocessing_function=preprocess_input
)

female_val_generator = train_datagen.flow_from_directory(
        "KDEFMF/Female",
        target_size=(targetx, targety),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=seed,
     subset="validation",
        
)

male_val_generator = train_datagen.flow_from_directory(
        "KDEFMF/Male",
        target_size=(targetx, targety),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=seed,
     subset="validation",
        
)

Found 286 images belonging to 7 classes.
Found 296 images belonging to 7 classes.


In [9]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
import time

def evaluate_keras_model(keras_model, data_generator, report_prefix="classification_report"):
    """
    This function takes a Keras model and a data generator as inputs, predicts the classes,
    and prints a classification report. The report variable changes with each function run.

    Args:
    keras_model (tensorflow.keras.Model): A compiled Keras model.
    data_generator (tf.keras.utils.Sequence): A Keras data generator for validation/testing.
    report_prefix (str): Prefix for the classification report variable name.

    Returns:
    dict: A dictionary containing the classification report with a unique key.
    """
    # Make predictions using the provided Keras model
    predictions = keras_model.predict(data_generator, steps=len(data_generator))
    
    # Get the predicted class indices
    y_pred = np.argmax(predictions, axis=1)

    # Generate a unique identifier for the report name
    timestamp = int(time.time())
    unique_cr_name = f"{report_prefix}_{timestamp}"

    # Print and return the classification report
    print('Classification Report')
    classification_report_result = classification_report(
        y_true=data_generator.classes,
        y_pred=y_pred,
        target_names=list(data_generator.class_indices.keys())
    )
    print(classification_report_result)

    # Store the classification report in a dictionary with a unique key
    results = {unique_cr_name: classification_report_result}
    
    return results

# Example setup:
models = [
    tf.keras.models.load_model('DNetKDEFMalemodel.keras'),
    tf.keras.models.load_model('MNV2KDEFMalemodel.keras'),
    tf.keras.models.load_model('RNKDEFMale.keras'),
    tf.keras.models.load_model('SeqKDEFMalemodel.keras'),
    tf.keras.models.load_model('IV3KDEFFemalemodel.keras')
    
]

# Assuming `data_generator` is already defined and is shared among models
data_generator = female_val_generator

# Loop through models and use the same data generator
for i, model in enumerate(models, start=1):
    report_prefix = f"report_model_{i}"
    results = evaluate_keras_model(model, data_generator, report_prefix=report_prefix)
    print(results)

6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 924ms/step
Classification Report
              precision    recall  f1-score   support

       angry       0.66      0.80      0.73        41
     disgust       0.80      0.68      0.74        41
        fear       0.79      0.52      0.63        42
       happy       0.93      0.98      0.95        41
     neutral       0.88      0.90      0.89        41
         sad       0.67      0.78      0.72        40
    surprise       0.83      0.88      0.85        40

    accuracy                           0.79       286
   macro avg       0.79      0.79      0.79       286
weighted avg       0.80      0.79      0.79       286

{'report_model_1_1728534120': '              precision    recall  f1-score   support\n\n       angry       0.66      0.80      0.73        41\n     disgust       0.80      0.68      0.74        41\n        fear       0.79      0.52      0.63        42\n       happy       0.93      0.98      0.95        41\n     neutral       0.88      0.90

In [10]:
Y_pred = DNetKDEFMale.predict(male_val_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true=male_val_generator.classes

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true,y_pred)
cm

labels=['angry','disgust','fear','happy','neutral','sad','surprise']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()

NameError: name 'DNetKDEFMale' is not defined

In [ ]:
Y_pred = DNetKDEFFemale.predict(male_val_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true=male_val_generator.classes

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true,y_pred)
cm

labels=['angry','disgust','fear','happy','neutral','sad','surprise']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()